In [1]:
import utils.utils as utils
import torch
from torch.optim import LBFGS
from torch.autograd import Variable
import numpy as np
import os
import cv2



In [2]:
def build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    target_content_representation = target_representations[0]
    target_style_representation = target_representations[1]

    current_set_of_feature_maps = neural_net(optimizing_img)

    current_content_representation = current_set_of_feature_maps[content_feature_maps_index].squeeze(axis=0)
    content_loss = torch.nn.MSELoss(reduction='mean')(target_content_representation, current_content_representation)

    style_loss = 0.0
    current_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(current_set_of_feature_maps) if cnt in style_feature_maps_indices]
    for gram_gt, gram_hat in zip(target_style_representation, current_style_representation):
        style_loss += torch.nn.MSELoss(reduction='sum')(gram_gt[0], gram_hat[0])
    style_loss /= len(target_style_representation)

    tv_loss = utils.total_variation(optimizing_img)

    total_loss = config['content_weight'] * content_loss + config['style_weight'] * style_loss + config['tv_weight'] * tv_loss

    return total_loss, content_loss, style_loss, tv_loss

In [3]:
def make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    # Builds function that performs a step in the tuning loop
    def tuning_step(optimizing_img):
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config)
        # Computes gradients
        total_loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        return total_loss, content_loss, style_loss, tv_loss

    # Returns the function that will be called inside the tuning loop
    return tuning_step

In [4]:
def neural_style_transfer(config):
    content_images_dir = os.path.join(os.path.abspath('data') , 'content-images')
    style_images_dir = os.path.join(os.path.abspath('data'), 'style-images')
    output_img_dir = os.path.join(os.path.abspath('data'), 'output-images')
    img_format = (4, '.jpg')  # saves images in the format: %04d.jpg

    content_img_path = os.path.join(content_images_dir, config['content_img_name'])
    style_img_path = os.path.join(style_images_dir,config['style_img_name'] )

    out_dir_name = 'combined_' + os.path.split(content_img_path)[1].split('.')[0] + '_' + os.path.split(style_img_path)[1].split('.')[0]
    dump_path = os.path.join(output_img_dir, out_dir_name)
    os.makedirs(dump_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    content_img = utils.prepare_img(content_img_path, 400, device)
    style_img = utils.prepare_img(style_img_path, 400, device)

    init_img = content_img
    optimizing_img = Variable(init_img, requires_grad=True)

    neural_net, content_feature_maps_index_name, style_feature_maps_indices_names = utils.prepare_model('vgg19', device)
    print(f'Using vgg19 in the optimization procedure.')

    content_img_set_of_feature_maps = neural_net(content_img)
    style_img_set_of_feature_maps = neural_net(style_img)

    target_content_representation = content_img_set_of_feature_maps[content_feature_maps_index_name[0]].squeeze(axis=0)
    target_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(style_img_set_of_feature_maps) if cnt in style_feature_maps_indices_names[0]]
    target_representations = [target_content_representation, target_style_representation]

    num_of_iterations_lbfgs = 1000

    optimizer = LBFGS((optimizing_img,), max_iter=num_of_iterations_lbfgs, line_search_fn='strong_wolfe')
    cnt = 0

    def closure():
        nonlocal cnt
        if torch.is_grad_enabled():
            optimizer.zero_grad()
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
        if total_loss.requires_grad:
            total_loss.backward()
        with torch.no_grad():
            print(f'L-BFGS | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
            utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, num_of_iterations_lbfgs, should_display=False)

        cnt += 1
        return total_loss

    optimizer.step(closure)

    return dump_path

In [5]:
if __name__ == "__main__":
    resource_path=os.path.abspath('data')
    content_image_dir=os.path.join(resource_path,'content-images')
    content_image=os.path.join(content_image_dir,'figures.jpg')
    style_image_dir=os.path.join(resource_path,'style-images')
    style_image=os.path.join(style_image_dir,'vg_starry_night.jpg')
    output_image_dir=os.path.join(resource_path, 'output-images')
    
    optimization_config = {
        'img_format': (4, '.jpg'),
        'content_img_name': 'example3.jpg',
        'style_img_name': 'example3.jpg',
        'height': 400,
        'content_weight': 1e5,
        'style_weight': 3e4,
        'tv_weight': 1e0,
        'optimizer': 'lbfgs',
        'model': 'vgg19',
        'init_method': 'content',
        'saving_freq': -1,
    }
    results_path = neural_style_transfer(optimization_config)
    
    
    

    
    

C:\Users\Saransh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Saransh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using vgg19 in the optimization procedure.
L-BFGS | iteration: 000, total loss=7021444726784.0000, content_loss=      0.0000, style loss=7021439040000.0000, tv loss=5514378.0000
L-BFGS | iteration: 001, total loss=7021443153920.0000, content_loss=      0.0001, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | iteration: 002, total loss=7021443153920.0000, content_loss=      0.0002, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | iteration: 003, total loss=7021443153920.0000, content_loss=      0.0002, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | iteration: 004, total loss=7021443153920.0000, content_loss=      0.0002, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | iteration: 005, total loss=7021443153920.0000, content_loss=      0.0002, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | iteration: 006, total loss=7021443153920.0000, content_loss=      0.0002, style loss=7021437600000.0000, tv loss=5514379.0000
L-BFGS | ite

L-BFGS | iteration: 061, total loss=13308854272.0000, content_loss=2171297070.3125, style loss=11092173750.0000, tv loss=45384160.0000
L-BFGS | iteration: 062, total loss=12858276864.0000, content_loss=2173074804.6875, style loss=10639672500.0000, tv loss=45530104.0000
L-BFGS | iteration: 063, total loss=12502904832.0000, content_loss=2171497460.9375, style loss=10285852500.0000, tv loss=45554624.0000
L-BFGS | iteration: 064, total loss=12270326784.0000, content_loss=2176783984.3750, style loss=10047855000.0000, tv loss=45687832.0000
L-BFGS | iteration: 065, total loss=11956096000.0000, content_loss=2177109179.6875, style loss=9733212187.5000, tv loss=45775340.0000
L-BFGS | iteration: 066, total loss=11760742400.0000, content_loss=2189945507.8125, style loss=9524716875.0000, tv loss=46079872.0000
L-BFGS | iteration: 067, total loss=11472644096.0000, content_loss=2185722460.9375, style loss=9240904687.5000, tv loss=46016456.0000
L-BFGS | iteration: 068, total loss=11254056960.0000, cont

L-BFGS | iteration: 123, total loss=5597436928.0000, content_loss=2229253710.9375, style loss=3318004921.8750, tv loss=50177912.0000
L-BFGS | iteration: 124, total loss=5567331328.0000, content_loss=2229229492.1875, style loss=3287893593.7500, tv loss=50208312.0000
L-BFGS | iteration: 125, total loss=5542589440.0000, content_loss=2230471679.6875, style loss=3261844453.1250, tv loss=50273420.0000
L-BFGS | iteration: 126, total loss=5502068736.0000, content_loss=2230871289.0625, style loss=3220866093.7500, tv loss=50331872.0000
L-BFGS | iteration: 127, total loss=5469585408.0000, content_loss=2234327929.6875, style loss=3184775156.2500, tv loss=50482408.0000
L-BFGS | iteration: 128, total loss=5434413056.0000, content_loss=2232658398.4375, style loss=3151318828.1250, tv loss=50436320.0000
L-BFGS | iteration: 129, total loss=5404552192.0000, content_loss=2233095312.5000, style loss=3120987421.8750, tv loss=50469616.0000
L-BFGS | iteration: 130, total loss=5369437696.0000, content_loss=223

L-BFGS | iteration: 185, total loss=4272270848.0000, content_loss=2239330468.7500, style loss=1980099375.0000, tv loss=52840932.0000
L-BFGS | iteration: 186, total loss=4261129472.0000, content_loss=2239798242.1875, style loss=1968464531.2500, tv loss=52866848.0000
L-BFGS | iteration: 187, total loss=4249118976.0000, content_loss=2240468945.3125, style loss=1955741250.0000, tv loss=52908788.0000
L-BFGS | iteration: 188, total loss=4234958592.0000, content_loss=2240461718.7500, style loss=1941565312.5000, tv loss=52931424.0000
L-BFGS | iteration: 189, total loss=4221690112.0000, content_loss=2241086132.8125, style loss=1927602539.0625, tv loss=53001400.0000
L-BFGS | iteration: 190, total loss=4211758592.0000, content_loss=2239710351.5625, style loss=1919066835.9375, tv loss=52981248.0000
L-BFGS | iteration: 191, total loss=4202309120.0000, content_loss=2239917382.8125, style loss=1909363710.9375, tv loss=53028024.0000
L-BFGS | iteration: 192, total loss=4193600768.0000, content_loss=223

...

L-BFGS | iteration: 748, total loss=2941414912.0000, content_loss=2167964453.1250, style loss=716016738.2812, tv loss=57433480.0000
L-BFGS | iteration: 749, total loss=2940748032.0000, content_loss=2168001171.8750, style loss=715306406.2500, tv loss=57440168.0000
L-BFGS | iteration: 750, total loss=2940228608.0000, content_loss=2167698828.1250, style loss=715091250.0000, tv loss=57438368.0000
L-BFGS | iteration: 751, total loss=2939870464.0000, content_loss=2167745898.4375, style loss=714687187.5000, tv loss=57437420.0000
L-BFGS | iteration: 752, total loss=2939602176.0000, content_loss=2167736718.7500, style loss=714425156.2500, tv loss=57440272.0000
L-BFGS | iteration: 753, total loss=2939191808.0000, content_loss=2167598437.5000, style loss=714153574.2188, tv loss=57439736.0000
L-BFGS | iteration: 754, total loss=2938802944.0000, content_loss=2167280859.3750, style loss=714082617.1875, tv loss=57439476.0000
L-BFGS | iteration: 755, total loss=2938334464.0000, content_loss=2167200000

L-BFGS | iteration: 811, total loss=2916058368.0000, content_loss=2161741796.8750, style loss=696847968.7500, tv loss=57468784.0000
L-BFGS | iteration: 812, total loss=2915708416.0000, content_loss=2161730273.4375, style loss=696504726.5625, tv loss=57473632.0000
L-BFGS | iteration: 813, total loss=2915257344.0000, content_loss=2161596679.6875, style loss=696188847.6562, tv loss=57471864.0000
L-BFGS | iteration: 814, total loss=2915048704.0000, content_loss=2161544921.8750, style loss=696032402.3438, tv loss=57471156.0000
L-BFGS | iteration: 815, total loss=2914747904.0000, content_loss=2161477734.3750, style loss=695797851.5625, tv loss=57472440.0000
L-BFGS | iteration: 816, total loss=2914210816.0000, content_loss=2161309765.6250, style loss=695429179.6875, tv loss=57472024.0000
L-BFGS | iteration: 817, total loss=2913934848.0000, content_loss=2161254296.8750, style loss=695200312.5000, tv loss=57480192.0000
L-BFGS | iteration: 818, total loss=2913440512.0000, content_loss=2161150976

L-BFGS | iteration: 874, total loss=2894914816.0000, content_loss=2156306250.0000, style loss=681115195.3125, tv loss=57493448.0000
L-BFGS | iteration: 875, total loss=2894522112.0000, content_loss=2156266406.2500, style loss=680759414.0625, tv loss=57496248.0000
L-BFGS | iteration: 876, total loss=2894297344.0000, content_loss=2156053906.2500, style loss=680749394.5312, tv loss=57493968.0000
L-BFGS | iteration: 877, total loss=2893979648.0000, content_loss=2156071289.0625, style loss=680413593.7500, tv loss=57494976.0000
L-BFGS | iteration: 878, total loss=2893735936.0000, content_loss=2156046875.0000, style loss=680193632.8125, tv loss=57495656.0000
L-BFGS | iteration: 879, total loss=2893488640.0000, content_loss=2155966210.9375, style loss=680027460.9375, tv loss=57495040.0000
L-BFGS | iteration: 880, total loss=2893008896.0000, content_loss=2155811328.1250, style loss=679701386.7188, tv loss=57496008.0000
L-BFGS | iteration: 881, total loss=2892820224.0000, content_loss=2155539062

L-BFGS | iteration: 937, total loss=2877503488.0000, content_loss=2151363671.8750, style loss=668638359.3750, tv loss=57501648.0000
L-BFGS | iteration: 938, total loss=2877300224.0000, content_loss=2151308984.3750, style loss=668490117.1875, tv loss=57501284.0000
L-BFGS | iteration: 939, total loss=2877018112.0000, content_loss=2151185351.5625, style loss=668331503.9062, tv loss=57501304.0000
L-BFGS | iteration: 940, total loss=2876683520.0000, content_loss=2151157226.5625, style loss=668022773.4375, tv loss=57503524.0000
L-BFGS | iteration: 941, total loss=2876421632.0000, content_loss=2150948242.1875, style loss=667974140.6250, tv loss=57499160.0000
L-BFGS | iteration: 942, total loss=2876203008.0000, content_loss=2150937304.6875, style loss=667765429.6875, tv loss=57500280.0000
L-BFGS | iteration: 943, total loss=2875950592.0000, content_loss=2150954882.8125, style loss=667495195.3125, tv loss=57500536.0000
L-BFGS | iteration: 944, total loss=2875718400.0000, content_loss=2150896093

L-BFGS | iteration: 1000, total loss=2861854976.0000, content_loss=2146902148.4375, style loss=657450996.0938, tv loss=57501852.0000
L-BFGS | iteration: 1001, total loss=2861573632.0000, content_loss=2146810937.5000, style loss=657260273.4375, tv loss=57502400.0000
L-BFGS | iteration: 1002, total loss=2861260032.0000, content_loss=2146685156.2500, style loss=657074062.5000, tv loss=57500928.0000
L-BFGS | iteration: 1003, total loss=2861076736.0000, content_loss=2146541992.1875, style loss=657030761.7188, tv loss=57503988.0000
L-BFGS | iteration: 1004, total loss=2860709632.0000, content_loss=2146470507.8125, style loss=656739023.4375, tv loss=57500192.0000
L-BFGS | iteration: 1005, total loss=2860596736.0000, content_loss=2146463671.8750, style loss=656633203.1250, tv loss=57499920.0000
L-BFGS | iteration: 1006, total loss=2860344320.0000, content_loss=2146421875.0000, style loss=656422031.2500, tv loss=57500320.0000
L-BFGS | iteration: 1007, total loss=2860130048.0000, content_loss=21